In [1]:
import cv2


In [2]:
cap=cv2.VideoCapture(0)
while(True):
    ret,frame=cap.read()
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    if ret==False:
        continue

    cv2.imshow("Video Frame",frame)
    cv2.imshow("Gray Frame",gray_frame)

    key_presses=cv2.waitKey(1) & 0xFF
    if key_presses==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
#Read and show video stream,capture images
#detect faces and show bounding box
#Flatten the largest face image(gray scale image) and save in numpy array
#Repeat the above  for multiple people to generate training data
import cv2
import numpy as np

cap=cv2.VideoCapture(0)

#Face Detection
face_cascade=cv2.CascadeClassifier("D:/Data Science/openCV/FaceRecognition/haarcascade_frontalface_alt.xml")
skip=0
face_data=[]
dataset_path="D:/Data Science/openCV/FaceRecognition/"
file_name=input("Enter the name of person:")

while(True):
    ret,frame=cap.read()
    

    if ret==False:
        continue

    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    
    faces=face_cascade.detectMultiScale(frame,1.3,5)  ##1.3 is scaling parameter and 5 is number of neighbours
    faces=sorted(faces,key=lambda f:f[2]*f[3])
    # print(faces)

    for face in faces[-1:]:
        x,y,w,h= face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)  ##x and y are the corrdinates and w and h are width and height
        

        #Extract (Crop out the required face) : Region of interest
        offset=10
        face_section= frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        skip +=1
        if skip %10==0:
             face_data.append(face_section)
             print(len(face_data))
             
    cv2.imshow("Video Frame",frame)
    cv2.imshow("Face section",face_section)
    

    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed==ord('q'):
            break

#convert our face list array into numpy array
face_data=np.asarray(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

np.save(dataset_path+file_name+'.npy',face_data)
print("Data Save successfully"+dataset_path+file_name+'.npy')

cap.release()
cv2.destroyAllWindows()

Enter the name of person: Saloni


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [1]:
import cv2
import numpy as np

cap=cv2.VideoCapture(0)

#Face Detection
face_cascade=cv2.CascadeClassifier("D:/Data Science/openCV/FaceRecognition/haarcascade_frontalface_alt.xml")
skip=0
face_data=[]
dataset_path="D:/Data Science/openCV/FaceRecognition/"
file_name=input("Enter the name of person:")

while(True):
    ret,frame=cap.read()
    

    if ret==False:
        continue

    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    
    faces=face_cascade.detectMultiScale(frame,1.3,5)  ##1.3 is scaling parameter and 5 is number of neighbours
    faces=sorted(faces,key=lambda f:f[2]*f[3])
    # print(faces)

    for face in faces[-1:]:
        x,y,w,h= face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)  ##x and y are the corrdinates and w and h are width and height
        

        #Extract (Crop out the required face) : Region of interest
        offset=10
        face_section= frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        skip +=1
        if skip %10==0:
             face_data.append(face_section)
             print(len(face_data))
             
        cv2.imshow("Face section",face_section)  # Moved inside the loop
    

    cv2.imshow("Video Frame",frame)
    

    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed==ord('q'):
            break

#convert our face list array into numpy array
face_data=np.asarray(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)

np.save(dataset_path+file_name+'.npy',face_data)
print("Data Save successfully"+dataset_path+file_name+'.npy')

cap.release()
cv2.destroyAllWindows()


Enter the name of person: Prabakaran


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
(19, 30000)
Data Save successfullyD:/Data Science/openCV/FaceRecognition/Prabakaran.npy


In [1]:
# load the training data
# x-values are stored in the numpy arrays
# y-values we need to assign to each person
# use KNN to find the prediction of face
# map the predicted id to name of the user
# Display the predictions on screen-bounding box and name

import cv2
import numpy as np
import os

########## KNN CODE ##########
def distance(v1,v2):
    # Eucledian
    return np.sqrt(((v1-v2) ** 2).sum())

def knn(train,test,k=5):
    dist=[]

    for i in range(train.shape[0]):
        # Get the vector and label
        ix=train[i, :-1]
        iy=train[i, :-1]
        # Compute the distance from test point
        d=distance(test,ix)
        dist.append([d,iy])
     # Sort based on distance and get top k
    dk=sorted(dist, key=lambda x:x[0])[:k]
     # Retrieve only the labels
    labels=np.array(dk)[:, -1]

    # Get frequencies of each label
    output=np.unique(labels,return_count=True)
    # Find max frequency and corresponding label
    index=np.argmax(output[1])
    return output[0][index]
###############################################

cap=cv2.VideoCapture(0)

#Face Detection
face_cascade=cv2.CascadeClassifier("D:/Data Science/openCV/FaceRecognition/haarcascade_frontalface_alt.xml")
skip=0
face_data=[]
dataset_path="D:/Data Science/openCV/FaceRecognition/"
labels=[]
class_id=0  #labels for the given file
names={}  #mapping between id-name

# Data preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        # Create mapping btw class_id and name
        names[class_id]=fx[:-4]
        print("loaded",fx)
        data_item =np.load(dataset_path+fx)
        face_data.append(data_item)

    # Create labels for the class
        target =class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)

face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset)
print(face_labels)

trainset=np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)

# Testing
while True:
    ret,frame=cap.read()
    
    if ret==False:
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)

    for face in faces:
        x,y,w,h=face

    # Get the face ROI
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))

        out=knn(trainset,face_section.flatten())
 
    # Display on the screen and rectangle around it
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("faces",frame)

    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

    
        
        






loaded arif.npy
[[119 115 132 ...  34  33  29]
 [121 122 131 ...  35  33  29]
 [107 112 117 ...  38  40  33]
 ...
 [115 118 128 ...  32  35  26]
 [119 120 131 ...  31  33  27]
 [116 119 129 ...  29  32  24]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
(15, 30001)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (5, 2) + inhomogeneous part.

In [2]:
import cv2
import numpy as np
import os

########## KNN CODE ##########
def distance(v1, v2):
    # Euclidean distance
    return np.sqrt(((v1 - v2) ** 2).sum())

def knn(train, test, k=5):
    dist = []

    for i in range(train.shape[0]):
        # Get the vector and label
        ix, iy = train[i, :-1], train[i, -1]
        # Compute the distance from test point
        d = distance(test, ix)
        dist.append((d, iy))

    # Sort based on distance and get top k
    dk = sorted(dist, key=lambda x: x[0])[:k]
    # Retrieve only the labels
    labels = [label for _, label in dk]

    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    # Find max frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]
###############################################

cap = cv2.VideoCapture(0)

# Face Detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip = 0
face_data = []
dataset_path = "D:/Data Science/openCV/FaceRecognition/"
labels = []
class_id = 0  # labels for the given file
names = {}  # mapping between id-name

# Data preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        # Create mapping between class_id and name
        names[class_id] = fx[:-4]
        print("loaded", fx)
        data_item = np.load(dataset_path + fx)
        face_data.append(data_item)

        # Create labels for the class
        target = class_id * np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0).reshape((-1, 1))

print(face_dataset)
print(face_labels)

trainset = np.concatenate((face_dataset, face_labels), axis=1)
print(trainset.shape)

# Testing
while True:
    ret, frame = cap.read()

    if ret == False:
        continue
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)

    for face in faces:
        x, y, w, h = face

        # Get the face ROI
        offset = 10
        face_section = frame[y - offset:y + h + offset, x - offset:x + w + offset]
        face_section = cv2.resize(face_section, (100, 100))

        out = knn(trainset, face_section.flatten())

        # Display on the screen and rectangle around it
        pred_name = names[int(out)]
        cv2.putText(frame, pred_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2,
                    cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)

    cv2.imshow("faces", frame)

    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


loaded Ahmed.npy
loaded arif.npy
loaded Mudassir.npy
loaded Prabakaran.npy
[[153 161 160 ...  10  15   7]
 [148 159 159 ...  22  26  13]
 [158 173 173 ...  21  24  21]
 ...
 [109  81  64 ...  22  24  19]
 [180 181 179 ...  20  21  18]
 [ 84  61  56 ...  21  23  18]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [2.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]
 [3.]]
(98, 30001)
